# Réseaux Antagonistes Génératifs (GANs)

## 1. Introduction

Les Réseaux Antagonistes Génératifs (GANs) sont composés de deux réseaux neuronaux qui s'affrontent dans un processus d'apprentissage concurrentiel :

- **Le générateur** : produit des données pour tromper le discriminateur.
- **Le discriminateur** : évalue les données et tente de distinguer les fausses des vraies.

Ces deux réseaux s'entraînent simultanément :

- Le générateur améliore sa capacité à produire des données réalistes.
- Le discriminateur affine sa capacité à détecter les fausses données.

## 2. Fonctionnement

Le processus d'entraînement consiste à mélanger les fausses données générées par le générateur avec de vraies données et les transmettre au discriminateur.  
Ce dernier, étant un classificateur binaire, attribue :

- **1** : si l'image est réelle.
- **0** : si l'image est fausse.


## 3. Architecture des réseaux

### 3.1 Générateur

**Entrée** : Bruit aléatoire (vecteur de dimension **100×1**, suivant une loi normale centrée réduite).

**Couches** :

1. **Dense layer + Reshape** avec des convolutions:
   - Transformation du vecteur **100×1** en un tenseur **8×8×128**.
   - Opération :  $ y = Wz + b $ avec  W  de taille **(8192,100)**.
   - Activation : **ReLU** (remplace les valeurs négatives par 0).
   - Reshape : **8192×1** en **(8,8,128)**.

2. **Conv2D Transpose (Upsampling)** avec des convolutions  :
   - **(8,8,128) → (16,16,256)**
   - **(16,16,256) → (32,32,128)**
   - **(32,32,128) → (64,64,64)**

3. **Conv2D (Finale)** :
   - Transformation finale en **(64,64,3)** pour obtenir une image **RGB**.

**Sortie** : Image générée **G(z)**.



### 3.2 Discriminateur

**Entrée** : Images réelles $x$ et images générées $ G(z)$.

**Fonction** : Classificateur binaire (renvoie une probabilité d'appartenance de l'entrée aux données réelles).



## 4. Fonction de Coût

### 4.1 Générateur $G_\theta$

**Objectif** : Faire croire au discriminateur que l'image générée est réelle.

$L_g = max_{\theta} E_{z \sim N(0,1)}\left[\log(D_{\theta '}(G_{\theta}(z)))\right]$


- Si **$D_{\theta '}(G(z))$ = 0.9**, le discriminateur pense que l'image est réelle → le générateur trompe le discriminateur
- Si **$D_{\theta '}(G(z))$ = 0.5**, le générateur a besoin de plus d'entraînement


### 4.2 Discriminateur $D_{\theta '}$

**Objectif** : Distinguer les vraies images des fausses.

$
L_d = \max_{\theta'} E_{x \sim p_{\text{data}}} \left[\log D_{\theta'}(x)\right] + E_{z \sim N(0,1)} \left[\log (1 - D_{\theta'}(G_{\theta}(z)))\right]
$ . 


- **D(x) → 1** pour les images réelles.
- **D(G(z)) → 0** pour les images générées.


## 5. Descente de gradient

Selon le feedback du discriminateur on a recours à la descente de gradient suivant le calcul des fonctions $L_d$ et $L_g$. Ainsi :   
$ \theta (t+1) = \theta (t) + \alpha \frac{1}{N}\partial_{\theta} \sum_{i=1}^N \log{(D_{\theta}(G_{\theta}(z^i(t)))}$  
$\theta' (t+1) = \theta' (t) + \alpha \frac{1}{N}\partial_{\theta'} \sum_{i=1}^{N}( \log(D_{\theta'}(x^i)) + \log( 1-  D_{\theta'}(G_{\theta}(z^i(t)))))$

Ces formules se basent sur la loi des grand nombres qui donne une bonne approximation de l'espérance. Cette dernière nécessitant une infinité d'échantillons, on est obligés ainsi de passer par la loi des grands nombres dans la formule des gradients.